# **Mengakses Citra Sentinel-2 MSI Surface Reflectance**

In [ ]:
import ee

In [ ]:
# Autentikasi Google Earth Engine

ee.Authenticate()

In [ ]:
# Ganti 'ee-yourproject' sesuai dengan Earth Engine Project Anda

ee.Initialize(project='ee-yourproject')

In [ ]:
# Penentuan rentang waktu akuisisi citra sesuai keperluan

tgl_awal = '2024-06-01'
tgl_akhir = '2024-06-30'

In [ ]:
# Penentuan titik fokus citra yang akan ditampilkan
# Koordinat bisa didapatkan dari Google Map dalam format bujur-lintang

titik = ee.Geometry.Point([114.776874, -3.449037])

In [ ]:
# Mengakses koleksi Citra Sentinel-2 MSI surface reflectance
# Menggunakan rentcang waktu yang sudah ditentukan
# Persentase awan pada citra maksimum 20% (lte: less than or equal)

s2_col = (
    ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
    .filterDate(tgl_awal,tgl_akhir)
    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE',20))
)

In [ ]:
# Visualisasi RGB Sentinel-2 MSI

import geemap

rgb_vis = {'min': 0, 'max': 5000, 'bands': ['B12','B11','B4']}

s2_map = geemap.Map()
s2_map.centerObject(titik,12)
s2_map.add_layer(s2_col.median(), rgb_vis, 'Sentinel-2 RGB')
s2_map

**Mengakses Google Drive dan membuka shapefile**

In [ ]:
# Mengakses Google Drive
# Sesuaikan nama folder Anda, contoh di bawah nama foldernya adalah 'geebook'

from google.colab import drive

drive.mount('/content/gdrive')
path = '/content/gdrive/MyDrive/geebook'

In [ ]:
# Membuka shapefile dari Google Drive
# Shapefile wajib poligon dan memiliki sistem koordinat geografis (bukan UTM), dan tanpa atribut ZM
# Sesuaikan nama shapefile dan lokasi folder shapefile Anda di dalam Google Drive
# Contoh di bawah, 'Banjarbaru.shp' terletak di dalam folder 'vector', di dalam folder 'geebook'

import geopandas as gpd

region_shp = gpd.read_file(path + '/vector/Banjarbaru.shp')
region_shp.plot()

In [ ]:
# Mengkonversi shapefile menjadi Earth Engine geometry

import json

region_js = json.loads(region_shp.to_json())
region_fc = ee.FeatureCollection(region_js)
region = ee.Geometry(region_fc.geometry())

**Memilih band dan memotong citra**

In [ ]:
# Memilih band-band Sentinel-2 yang diperlukan

s2_image = s2_col.median().select('B2','B3','B4','B8','B11','B12')

In [ ]:
# Memotong Sentinel-2 menurut batas wilayah

s2_clipped = s2_image.clip(region)

In [ ]:
# Visualisasi RGB Sentinel-2 MSI yang sudah dipotong

s2_map = geemap.Map()
s2_map.centerObject(region,12)
s2_map.add_layer(s2_clipped, rgb_vis, 'Sentinel-2 RGB')
s2_map

**Mengekspor citra yang sudah dipotong ke dalam Google Drive**

In [ ]:
# Mengekspor Citra Sentinel-2 ke dalam Google Drive
# Citra akan tersimpan di dalam folder 'Citra' dengan nama 'S2_SR_image.tif'
# Resolusi spasial 10 meter dan Sistem Koordinat UTM WGS 1984 Zone 50S ('EPSG:32750')

task_s2 = ee.batch.Export.image.toDrive(
    image=s2_clipped,
    description='S2_SR_image',
    folder='Citra',
    scale=10,
    maxPixels=1e13,
    crs='EPSG:32750'
)

task_s2.start()

In [ ]:
# Mencek status ekspor citra ke dalam Google Drive
# Jika statusnya 'RUNNING' berarti proses ekspor belum selesai
# Jika statusnya 'COMPLETED' berarti proses ekspor sudah selesai
# Jika statusnya 'READY' berarti sudah ada citra dengan nama dan folder yang sama
# Anda dapat logout dari Google Colab atau mematikan komputer, proses ekspor akan terus berjalan sampai selesai

task_s2.status()